In [2]:
from http import client
import xlsxwriter
from openpyxl import Workbook, load_workbook
import pandas as pd
from bs4 import BeautifulSoup
import os
import requests
import win32com.client as win32 
from pathlib import Path

In [ ]:
# def append_df_to_excel(df, excel_path):
#     df_excel = pd.read_excel(excel_path)
#     result = pd.concat([df_excel, df], ignore_index=True)
#     result.to_excel(excel_path, index=False)

def clean_data(df):
    df = df.iloc[:, :2]
    df.columns = df.columns.droplevel([0,1]) #drop first two-level of column index
    df = df[df["投資公司名稱"].str.contains("備註") == False]
    df = pd.concat([df["投資公司名稱"], df["子公司名稱"]]).to_frame()
    df = df.replace(r' ','', regex=True)
    df = df.drop_duplicates(ignore_index=True).rename(columns={0:'company'})
    res = df[df["company"].str.contains("本公司") == False].reset_index()
    res = pd.concat([noSubsidiary_df['company'], res['company']]).to_frame()
    res['stock_num'] = stock_number
    res['查核事務所'] = ''
    res['Client Code'] = ''
    res['公司類型'] = ''
    res['關係'] = ''
    
    try:
        #save as team_clientcode_stockNo_companyName
        if res['stock_num'].iloc[0] in case_df['stock_num'].values:
            res = res.merge(case_df, on='stock_num')
            audit_team = res['team'].iloc[0].strip()
            client_code = res['client_code'].iloc[0].strip()
            res = res.drop_duplicates(ignore_index=True, keep='first')
            res = res[['company', '查核事務所', 'Client Code', '公司類型', '關係']]
            res.to_excel('Result/Deloitte_Client/'+audit_team+'_'+client_code+'_'+stock_number +'_'+ company_name +'.xlsx', index=False)
            
        elif res['stock_num'].iloc[0] not in case_df['stock_num'].values:
            pass

    except OSError:
        new_name = company_name.replace('?','_')

        if res['stock_num'].iloc[0] in case_df['stock_num'].values:
            res = res.merge(case_df, on='stock_num')
            audit_team = res['team'].iloc[0].strip()
            client_code = res['client_code'].iloc[0].strip()
            res = res.drop_duplicates(ignore_index=True, keep='first')
            res = res[['company', '查核事務所', 'Client Code', '公司類型', '關係']]
            res.to_excel('Result/Deloitte_Client/'+audit_team+'_'+client_code+'_'+stock_number +'_'+ new_name +'.xlsx', index=False)
            
        elif res['stock_num'].iloc[0] not in case_df['stock_num'].values:
            pass
            # res = res.drop_duplicates(ignore_index=True, keep='first')
            # res = res[['company', '查核事務所', 'Client Code', '公司類型', '關係']]
            # res.to_excel('Result/Others/'+ stock_number +'_'+ new_name +'.xlsx', index=False)

"""
IMPORT stock_num / Audit team / Client code INFO
"""
case_df = pd.read_csv('case_list/case_ls.csv')


"""
    Financial Report
"""
# tifrs-2021Q4
dir = 'C:/Users/Kylecchen/OneDrive - Deloitte (O365D)/桌面/QRM/tifrs_抓母子公司名/tifrs-2021Q4'

for filename in os.listdir(dir):
    if filename.endswith('.html'):
        fname = os.path.join(dir,filename)
        with open(fname, 'r', encoding="utf-8") as f:
            soup = BeautifulSoup(f,'html.parser')
            for body in soup("tbody"):
                body.unwrap()
                
            df = pd.read_html(str(soup), flavor="bs4")
            header = soup.find_all('div', class_='header')
            header_str = header[0].select_one("span.zh").text
            start_index = header_str.index(' ')
            end_index = header_str.index('2021年')
            company_name = header_str[start_index : end_index].strip()
            stock_number = header_str[:start_index]
            noSubsidiary_df = pd.DataFrame({'company':[company_name], 'stock_num':[stock_number]})
            noSubsidiary_df['查核事務所'] = ''
            noSubsidiary_df['Client Code'] = ''
            noSubsidiary_df['公司類型'] = ''
            noSubsidiary_df['關係'] = ''
            
            try:
                df_match = pd.read_html(str(soup), flavor="bs4", match='列入合併財務報表之子公司')[0]
                clean_data(df_match)
            except IndexError:
                df_match = pd.read_html(str(soup), flavor="bs4", match='列入合併財務報表之子公司')[1]
                clean_data(df_match)
            except KeyError:
                # continue
                print(company_name + " this company has Key error!!!")
            except ValueError:
                if noSubsidiary_df['stock_num'].iloc[0] in case_df['stock_num'].values:
                    noSubsidiary_df = noSubsidiary_df.merge(case_df, on='stock_num')
                    audit_team = noSubsidiary_df['team'].iloc[0].strip()
                    client_code = noSubsidiary_df['client_code'].iloc[0].strip()
                    noSubsidiary_df = noSubsidiary_df[['company', '查核事務所', 'Client Code', '公司類型', '關係']]
                    noSubsidiary_df.to_excel('Result/Deloitte_Client/'+audit_team+'_'+client_code+'_'+stock_number +'_'+ company_name +'.xlsx', index=False)
                    
                    
                elif noSubsidiary_df['stock_num'].iloc[0] not in case_df['stock_num'].values:
                    pass
                    # noSubsidiary_df = noSubsidiary_df[['company', '查核事務所', 'Client Code', '公司類型', '關係']]
                    # noSubsidiary_df.to_excel('Result/Others/'+ stock_number +'_'+ company_name +'.xlsx', index=False)
     

#### VBA

In [ ]:
#paste "集團公費邏輯指標 3-1 非審計服務公費說明" sheet to each file
excel = win32.gencache.EnsureDispatch('Excel.Application')

# excel can be visible or not
excel.Visible = False

# open workbooks
f_path = Path.cwd()
f_path2 = f_path /'Result/Deloitte_Client/' 

template_f = '2330_TAI080_台灣積體電路製造股份有限公司.xlsx'
template = f_path / template_f

for filename in os.listdir(f_path2):
    fname = os.path.join(f_path2,filename)

    wb1 = excel.Workbooks.Open(template)
    wb2 = excel.Workbooks.Open(fname)
    sheetname = "集團公費邏輯指標 3-1 非審計服務公費說明"
    wb1.Worksheets(sheetname).Copy(Before=wb2.Sheets(1))

    # remove hashtag to save and quit modified Excel file
    wb2.Save()
    wb2.Close(True)
excel.Quit()

In [19]:
#copy and paste drop down list (data validation) to each file
excel = win32.gencache.EnsureDispatch('Excel.Application')

# excel can be visible or not
excel.Visible = True  # False

# open workbooks
f_path = Path.cwd()
f_path2 = f_path /'Result/Deloitte_Client/' 

template_f = '2330_TAI080_台灣積體電路製造股份有限公司.xlsx'
template = f_path / template_f

for filename in os.listdir(f_path2):
    fname = os.path.join(f_path2,filename)

    wb_template = excel.Workbooks.Open(template)
    wb_f = excel.Workbooks.Open(fname)

    temp_sheet = wb_template.Worksheets("Sheet1")
    f_sheet = wb_f.Worksheets("Sheet1")

    rowCnt_f = f_sheet.UsedRange.Rows.Count
    
    for i in range(2, rowCnt_f+1):
        # print(i)
        temp_sheet.Range("B2:E2").Copy()
        f_sheet.Range("B%s:E%s"%(i,i)).PasteSpecial(Paste=-4104)
        
    
    wb_f.Save()
    wb_f.Close(True)
    
excel.Quit()

In [23]:
#set column width
excel = win32.gencache.EnsureDispatch('Excel.Application')

# excel can be visible or not
excel.Visible = False  # False

# open workbooks
f_path = Path.cwd()
f_path2 = f_path /'grouped/組別待確認/' 

# template_f = '2330_TAI080_台灣積體電路製造股份有限公司.xlsx'
# template = f_path / template_f

for filename in os.listdir(f_path2):
    fname = os.path.join(f_path2,filename)

    # wb1 = excel.Workbooks.Open(template)
    wb = excel.Workbooks.Open(fname)
    ws = wb.Worksheets('Sheet1')
    ws1 = wb.Worksheets("集團公費邏輯指標 3-1 非審計服務公費說明")
    
    ws.Range("A:A").ColumnWidth = 100
    ws.Range("B:E").ColumnWidth = 15
    ws.Range("A:E").Font.Name = "Calibri"
    ws1.Range("A:B").Font.Name = "Calibri"


    wb.Save()
    wb.Close(True)
excel.Quit()